In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import fonctions
from fonctions import scan_xml, modif_chemins_declaration, id_table

# options
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

# dossier
folder = 'D:/Fichiers_dev/test_xml/tables/'
folder2 = 'D:/Fichiers_dev/test_xml/'

data3 = pd.read_csv(folder + 'data_nettoyees.csv')

In [ ]:
# les chemins qui vont déterminer les champs de la table mandat électif sont les chemins contenant mandat électif
# mais pas montant, cela sera pour la table des ressources
# concretement voici le code
mask_champs_mandatElectifDto = (data3['chemins'].str.contains('mandatElectifDto/items/', na=False)) & ~(data3['chemins'].str.contains('montant/', na=False))

champs_mandatElectifDto = data3[mask_champs_mandatElectifDto]['chemins'].unique()
champs_mandatElectifDto

# creation de la table
mandatElectifDto = data3[data3['chemins'].isin(champs_mandatElectifDto)][['id_declaration', 'champs', 'valeurs']]

# encodage id_declaration
mandatElectifDto['id_declaration'] = encoder_id_declaration.transform(mandatElectifDto['id_declaration'])

# ajout de l'id_mandat
ids_mandat1 = []
champs_mandat1 = []

# creation de l'id_mandat, qui s'incrémente à chaque nouveau mandat, chaque nouveau id dans le tableau
for element in mandatElectifDto['champs'].values:
    champs_mandat1.append(element)
    i = f'm{champs_mandat1.count("id") - 1}'
    ids_mandat1.append(i)

mandatElectifDto['id_mandat'] = ids_mandat1

# supression des doublons
mandatElectifDto = mandatElectifDto.drop(mandatElectifDto[mandatElectifDto.duplicated(subset=['id_declaration','champs', 'id_mandat'],
                                                                                                      keep='last')].index)

mandatElectifDto = mandatElectifDto.pivot(index=['id_declaration', 'id_mandat'],
                                          columns='champs', values='valeurs').reset_index()


# export en csv
mandatElectifDto.to_csv(folder_export + 'mandatElectifDto.csv', index=False)